<h1 style='font-size:40px'> Loading and Preprocessing Data with TensorFlow</h1>

<h2 style='font-size:30px'> The Data API</h2>
<div> 
    <ul style='font-size:20px'>
        <li> 
            A Data API é um módulo do Tensor Flow voltado ao tratamento de datasets volumosos. É interessante ser usado quando os dados não cabem na memória RAM ou placa de vídeo.
        </li>
        <li> 
            A classe `Dataset` é onde nossos dados são armazenados.
        </li>
    </ul>
</div>

In [9]:
import tensorflow as tf
import tensorflow.keras as keras

# Criando um `Dataset` a partir de um `tf.range`. 
X = tf.range(10)

# Cada número de `X` será encapsulado em um tensor.
dataset = tf.data.Dataset.from_tensor_slices(X)

for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [73]:
# O mesmo Dataset poderia ter sido montado da seguinte maneira:
dataset = tf.data.Dataset.range(10)

<h3 style='font-size:30px;font-style:italic'> Chaining Transformations</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            As funções da classe `Dataset` nunca fazem transformações in-place; sempre retornam um novo objeto.
        </li>
    </ul>
</div>

In [74]:
#  `repeat` repetirá os dados do dataset, enquanto `batch` vai criar batches com n instâncias.
for item in dataset.repeat(3).batch(7):
    # Para evitar que o último batch com os elementos restantes do dataset seja formado, passe `drop_remainder`=True.
    print(item)
    
# Observe que `batch` não embaralha os dados na hora de sua separação.

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [39]:
# `map` aplica uma função por elemento.
# `apply` invoca uma transformação a todo o dataset.
list(dataset.map(lambda x: x**2))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=16>,
 <tf.Tensor: shape=(), dtype=int64, numpy=81>,
 <tf.Tensor: shape=(), dtype=int64, numpy=256>,
 <tf.Tensor: shape=(), dtype=int64, numpy=625>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1296>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2401>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4096>,
 <tf.Tensor: shape=(), dtype=int64, numpy=6561>]

In [47]:
# `filter`, é bastante eficaz em remover dados indesejados.
list(dataset.filter(lambda x: x<5))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>]

<h3 style='font-size:30px;font-style:italic'> Shuffling the Data</h3>
<div> 
    <ul style='font-size:20px'>
        <li> 
            O método `shuffle` embaralha as instâncias com a seguinte lógica: escolhe as primeiras x instâncias do dataset e as agrupa em um buffer. Daí, as mistura e sorteia um dado, quando solicitado. Após a extração, o buffer fica com um espaço sobrando, que é preenchido com a próximoa instância do dataset.
        </li>
        <li> 
            Por conta do algoritmo, é contraindicado usar `buffer_size`'s pequenos. A documentação do TF indica, até mesmo, designarmos um valor maior ou igual ao o dataset.
        </li>
    </ul>
</div>

In [70]:
# Embaralhando um dataset artificial (`buffer_size`=10).

# `reshuffle_each_iteration` garante que um novo embaralhamento dos dados ocorrerá, caso usemos `repeat`, por exemplo.
dataset = tf.data.Dataset.range(5)
list(dataset.shuffle(10, seed=42, reshuffle_each_iteration=True).repeat(2))

[<tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>,
 <tf.Tensor: shape=(), dtype=int64, numpy=3>,
 <tf.Tensor: shape=(), dtype=int64, numpy=4>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>,
 <tf.Tensor: shape=(), dtype=int64, numpy=0>,
 <tf.Tensor: shape=(), dtype=int64, numpy=2>]

<h4 style='font-size:30px;font-style:italic;text-decoration:underline'> Interleaving lines from multiple files</h4>

<p style='color:red'> Explicar o Interleaving lines from multiple files</p>